<a href="https://colab.research.google.com/github/futartup/S8-assignment/blob/master/transform_train_test_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import glob
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np


# **Create transformation pipeline for training**

In [0]:
import albumentations as A
from albumentations.pytorch import ToTensor

pytorch_train_transform = transforms.Compose([
                        transforms.RandomRotation((-7.0, 7.0)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])

pytorch_test_transform = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])


class AlbumentationDataSet():
  def __init__(self, train=True):
    if train:
      self.albu_aug = A.Compose(
                      [
                          A.HorizontalFlip(p = 0.2), # apply horizontal flip to 50% of images
                          #A.VerticalFlip(always_apply=True, p=0.5),
                          A.RandomRotate90(),
                          #A.RandomContrast(always_apply=False, p=0.1, limit=(-0.25999999046325684, 0.17000000178813934)), # apply random contrast
                          #A.RandomGamma(always_apply=False, p=0.1, gamma_limit=(62, 105)), # apply random gamma
                          #A.RandomBrightness(p=0.1), # apply random brightness
                          #A.GaussNoise(p=1.0),
                          #A.RGBShift(),
                          #A.GaussianBlur(),
                          A.ShiftScaleRotate(always_apply=False, 
                                             p=0.2, 
                                             shift_limit=(-0.07000000029802322, 0.05999999865889549), 
                                             scale_limit=(-0.10000000149011612, 0.05000000074505806), 
                                             rotate_limit=(-22, 12), 
                                             interpolation=0, border_mode=0
                                             ),
                          # A.OneOf([
                          #     A.GridDistortion(p=0.1),
                          # ], p=0.2),
                          # A.OneOf([
                          #     A.MotionBlur(p=0.2),
                          #     A.MedianBlur(blur_limit=3, p=0.1),
                          #     A.Blur(blur_limit=3, p=0.1),
                          # ], p=0.2),
                          A.Cutout(num_holes=2, max_h_size=10, max_w_size=10, always_apply=True, p=1.0),
                          A.Normalize(
                            mean=[0.5, 0.5, 0.5],
                            std=[0.5, 0.5, 0.5],
                            always_apply=True,
                            ),
                          
                          ToTensor(), # convert the image to PyTorch tensor
                      ]
                  )
    else:
        self.albu_aug = A.Compose([
            A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], always_apply=True),
            ToTensor(),
        ])
      
  def __call__(self, image):
    # image = plt.imread(numpy.asarray(self.image_list[i][0]))
    # image = Image.fromarray(image).convert('RGB')
    # image = self.albu_aug(image=np.array(image))['image']
    # #image = np.transpose(image, (2, 0, 1)).astype(np.float32)
    # return image
    image = np.array(image)
    image = self.albu_aug(image=image)['image']
    return image
  

In [0]:
# transform_train = transforms.Compose([augmentation_pipeline,
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])
#transform_train = augmentation_pipeline
# Get the image list


# **Apply Transformations on the test set**

In [0]:
# Normalize the test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# **Get the loader for the train data**

In [0]:
def get_train_loader():
  trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                          download=True, transform=AlbumentationDataSet())
  #alb_dataset = AlbumentationDataSet(image_list=trainset)
  #train_loader = DataLoader(dataset=alb_dataset, batch_size=64, shuffle=True, num_workers=2)
  train_loader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)
  return train_loader


# **Get the loader for the test data**

In [0]:
def get_test_loader():  
  testset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                          download=True, transform=AlbumentationDataSet(False))
  test_loader = DataLoader(testset, batch_size=64,
                           shuffle=False, num_workers=2)
  return test_loader

In [0]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')